In [ ]:
!pip install import_ipynb
!pip install segmentation-models
!pip install tensorflow==2.8
!pip install keras==2.0

In [ ]:
import import_ipynb
%run /content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/UpdatedMeanIoU.ipynb
#import unet
#from unet import build_unet

In [ ]:
#from unet import build_unet
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.metrics import MeanIoU
from keras.callbacks import EarlyStopping
import tensorflow as tf
import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()

Segmentation Models: using `keras` framework.


'tf.keras'

In [ ]:

train_frame_list = os.listdir("/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/train_frame/frame")
train_frame_list = sorted(train_frame_list)
print(train_frame_list)
print(len(train_frame_list))

train_mask_list = os.listdir("/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/train_mask/mask")
train_mask_list = sorted(train_mask_list)
print(train_mask_list)
print(len(train_mask_list))

for frame, mask in zip(train_frame_list, train_mask_list):
  frame = frame.split(".")
  mask = mask.split(".")
  if frame[0]!=mask[0]:
    print(frame[0], mask[0])
    break

In [ ]:
'''
file_path = "/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/train_mask/mask/video_03_000009000 (1).png"
# Check if the file exists before attempting to delete it
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"The file {file_path} has been deleted.")
else:
    print(f"The file {file_path} does not exist.")
'''

In [ ]:

val_frame_list = os.listdir("/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/val_frame/frame")
val_frame_list = sorted(val_frame_list)
print(val_frame_list)
print(len(val_frame_list))

val_mask_list = os.listdir("/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/val_mask/mask")
val_mask_list = sorted(val_mask_list)
print(val_mask_list)
print(len(val_mask_list))

for frame, mask in zip(val_frame_list, val_mask_list):
  frame = frame.split(".")
  mask = mask.split(".")
  if frame[0]!=mask[0]:
    print(frame[0], mask[0])


In [ ]:
'''
file_path = "/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/val_frame/frame/video_38_12900.jpg"
# Check if the file exists before attempting to delete it
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"The file {file_path} has been deleted.")
else:
    print(f"The file {file_path} does not exist.")
'''

In [ ]:

test_frame_list = os.listdir("/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/test_frame/frame")
test_frame_list = sorted(test_frame_list)
print(test_frame_list)
print(len(test_frame_list))

test_mask_list = os.listdir("/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/test_mask/mask")
test_mask_list = sorted(test_mask_list)
print(test_mask_list)
print(len(test_mask_list))

for frame, mask in zip(test_frame_list, test_mask_list):
  frame = frame.split(".")
  mask = mask.split(".")
  if frame[0]!=mask[0]:
    print(frame[0], mask[0])


In [1]:
'''
file_path = "/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/test_frame/frame/video_38_12900.jpg"
# Check if the file exists before attempting to delete it
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"The file {file_path} has been deleted.")
else:
    print(f"The file {file_path} does not exist.")
'''

'\nfile_path = "/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/test_frame/frame/video_38_12900.jpg"\n# Check if the file exists before attempting to delete it\nif os.path.exists(file_path):\n    os.remove(file_path)\n    print(f"The file {file_path} has been deleted.")\nelse:\n    print(f"The file {file_path} does not exist.")\n'

In [ ]:
#New generator with rotation and shear where interpolation that comes with rotation and shear are thresholded in masks.
#This gives a binary mask rather than a mask with interpolated values.
seed=24
batch_size= 8
n_classes= 9

In [ ]:
img_data_gen_args = dict(rescale = 1/255., #Original pixel values are 0 and 255. So rescaling to 0 to 1
                      width_shift_range=0.3,
                      height_shift_range=0.3,
                      shear_range=0.5,
                      horizontal_flip=True,
                      vertical_flip=True,
                      fill_mode='reflect')

In [ ]:
#If You need to resize images then add this to the flow_from_directory parameters
#target_size=(150, 150), #Or whatever the size is for your network

image_data_generator = ImageDataGenerator(**img_data_gen_args)
image_generator = image_data_generator.flow_from_directory("/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/train_frame/",
                                                           seed=seed,
                                                           batch_size=batch_size,
                                                           class_mode=None)  #Very important to set this otherwise it returns multiple numpy arrays
                                                                            #thinking class mode is binary.

mask_generator = image_data_generator.flow_from_directory("/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/train_mask/",
                                                         seed=seed,
                                                         batch_size=batch_size,
                                                         color_mode = 'grayscale',   #Read masks in grayscale
                                                         class_mode=None)


valid_img_generator = image_data_generator.flow_from_directory("/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/val_frame/",
                                                               seed=seed,
                                                               batch_size=batch_size,
                                                               class_mode=None) #Default batch size 32, if not specified here
valid_mask_generator = image_data_generator.flow_from_directory("/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/val_mask/",
                                                               seed=seed,
                                                               batch_size=batch_size,
                                                               color_mode = 'grayscale',   #Read masks in grayscale
                                                               class_mode=None)  #Default batch size 32, if not specified here




Found 200 images belonging to 1 classes.
Found 200 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.


In [ ]:
test_img_generator = image_data_generator.flow_from_directory("/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/test_frame/",
                                                              seed=seed,
                                                              batch_size=32,
                                                              class_mode=None) #Default batch size 32, if not specified here

test_mask_generator = image_data_generator.flow_from_directory("/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/test_mask/",
                                                              seed=seed,
                                                              batch_size=32,
                                                              color_mode = 'grayscale',   #Read masks in grayscale
                                                              class_mode=None)  #Default batch size 32, if not specified here


Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.


In [ ]:
from keras import backend as K

#Dice metric can be a great metric to track accuracy of semantic segmentation.
def dice_metric(y_pred, y_true):
    intersection = K.sum(K.sum(K.abs(y_true * y_pred), axis=-1))
    union = K.sum(K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1))
    # if y_pred.sum() == 0 and y_pred.sum() == 0:
    #     return 1.0

    return 2*intersection / union


In [ ]:
# preprocess input
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)
image_generator = preprocess_input(image_generator)
mask_generator = preprocess_input(mask_generator)
valid_img_generator = preprocess_input(valid_img_generator)
valid_mask_generator = preprocess_input(valid_mask_generator)
test_img_generator = preprocess_input(test_img_generator)
test_mask_generator = preprocess_input(test_mask_generator)

train_generator = zip(image_generator,mask_generator)
val_generator = zip(valid_img_generator, valid_mask_generator)
test_generator = zip(test_img_generator, test_mask_generator)

model1 = sm.Unet(BACKBONE, encoder_weights='imagenet')
model2 = sm.Linknet(BACKBONE, encoder_weights='imagenet')

In [ ]:
mIOU = UpdatedMeanIoU(num_classes=9)

num_train_imgs = len(os.listdir('/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/train_frame/frame/'))
num_train_mask = len(os.listdir('/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/train_mask/mask/'))
print(num_train_imgs, num_train_mask)
steps_per_epoch = num_train_imgs //batch_size
print(steps_per_epoch)


model1.compile(optimizer=Adam(lr = 1e-3), loss='categorical_crossentropy', metrics=[mIOU, dice_metric])
early_stopping = EarlyStopping(monitor="val_loss", patience=20, verbose=1, restore_best_weights=True,)

history1=model1.fit(train_generator, validation_data=val_generator, steps_per_epoch=steps_per_epoch,
                  validation_steps=steps_per_epoch, epochs=1,  callbacks=[early_stopping],)

In [ ]:
model2.compile(optimizer=Adam(lr = 1e-3), loss='categorical_crossentropy', metrics=[mIOU, dice_metric])

history2=model2.fit(train_generator, validation_data=val_generator, steps_per_epoch=steps_per_epoch,
                  validation_steps=steps_per_epoch, epochs=1,  callbacks=[early_stopping],)

In [ ]:
model3 = sm.Linknet("efficientnetb7", encoder_weights='imagenet')
model3.compile(optimizer=Adam(lr = 1e-3), loss='categorical_crossentropy', metrics=[mIOU, dice_metric])
history3=model3.fit(train_generator, validation_data=val_generator, steps_per_epoch=steps_per_epoch,
                  validation_steps=steps_per_epoch, epochs=1,  callbacks=[early_stopping],)

In [ ]:
print(test_img_generator[0].shape)
num_test_imgs = len(test_img_generator[0])

print(num_test_imgs)
batch_size = 16

steps_per_epoch = num_test_imgs //batch_size
print(steps_per_epoch)

_, dice_metric, mIOU = model1.evaluate(test_generator, steps=steps_per_epoch)
print("Dice metric is = ", (dice_metric * 100.0), "%", "mIOU is = ", (mIOU * 100.0), "%")

_, dice_metric, mIOU = model2.evaluate(test_generator, steps=steps_per_epoch)
print("Dice metric is = ", (dice_metric * 100.0), "%", "mIOU is = ", (mIOU * 100.0), "%")


_, dice_metric, mIOU = model3.evaluate(test_generator, steps=steps_per_epoch)
print("Dice metric is = ", (dice_metric * 100.0), "%", "mIOU is = ", (mIOU * 100.0), "%")

#we can ensemble multiple models

In [ ]:
#model.save('semanticSegmentation_50epochs.hdf5')
#model = tf.keras.models.load_model("semanticSegmentation_50epochs.hdf5", compile=False)